> Nota: Esta versión ya tiene un entendimiento más profundo de los pasos requeridos del algoritmo, pero falta entender la teoría de los autovectores

> Nota: Versión 3 implenta el código hasta covarianza. Versión 4 incluye el entendimiento de los autovectores. v5 el código correspondiente. v6 debería incluir transform y todos felices.

Según [Kothe (2011)](https://www.youtube.com/playlist?list=PLbbCsk7MUIGcO_lZMbyymWU2UezVHNaMq), el algoritmo Common Spatial Pattern (CSP) resuelve el siguiente problema, llamado también [spatial filter problem](https://www.youtube.com/watch?v=S4znknOIcRk):

$$S = WA$$

Donde W es el resultado del algoritmo CSP (que al parecer proviene de la rama de procesamiento de señales). $A$ es la matriz de entrada (que al parecer tiene más dimensiones de las que pensaba - channels $\times$ samples), y S es la matriz de salida (que debería ser sources $\times$ samples).

Fatal: usa imágenes

El problema de filtros espaciales puede resumirse de la siguiente forma: Teniendo una fuente de información ($A$), ya sea a través de sensores, micrófonos, electrodos, etc; se quiere hallar las fuentes de esas señales ($S$). Debido a razones biológicas, los sensores no captan los lugares precisos de donde se generan estas señales. Así, un electrodo ubicado en la parte derecha de la cabeza puede captar con más fuerza una señal generada dentro del hemisferio izquierdo del cerebro que un electrodo de ubicado justo por encima de esa zona, dependiendo de diversos factores.

Por eso voy a analizar linea por línea el código de la [función fit de mne](https://github.com/mne-tools/mne-python/blob/maint/0.19/mne/decoding/csp.py#L144-L255).

# Implementción MNE de Common Spatial Pattern

Los parámetros del algoritmo/objeto CSP, según la [documentación](https://github.com/mne-tools/mne-python/blob/maint/0.19/mne/decoding/csp.py#L144-L255), son:

        Parameters
        ----------
        n_components : int, default 4
            The number of components to decompose M/EEG signals.
            This number should be set by cross-validation.
        reg : float | str | None (default None)
            If not None (same as ``'empirical'``, default), allow
            regularization for covariance estimation.
            If float, shrinkage is used (0 <= shrinkage <= 1).
            For str options, ``reg`` will be passed to ``method`` to
            :func:`mne.compute_covariance`.
        log : None | bool (default None)
            If transform_into == 'average_power' and log is None or True, then
            applies a log transform to standardize the features, else the features
            are z-scored. If transform_into == 'csp_space', then log must be None.
        cov_est : 'concat' | 'epoch', default 'concat'
            If 'concat', covariance matrices are estimated on concatenated epochs
            for each class.
            If 'epoch', covariance matrices are estimated on each epoch separately
            and then averaged over each class.
        transform_into : {'average_power', 'csp_space'}
            If 'average_power' then self.transform will return the average power of
            each spatial filter. If 'csp_space' self.transform will return the data
            in CSP space. Defaults to 'average_power'.
        norm_trace : bool
            Normalize class covariance by its trace. Defaults to False. Trace
            normalization is a step of the original CSP algorithm [1]_ to eliminate
            magnitude variations in the EEG between individuals. It is not applied
            in more recent work [2]_, [3]_ and can have a negative impact on
            patterns ordering.
        cov_method_params : dict | None
            Parameters to pass to :func:`mne.compute_covariance`.
            
Y la inicialización en el [tutorial](https://mne.tools/stable/auto_examples/decoding/plot_decoding_csp_eeg.html) es:

        csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)

Lo cual nos deja con ciertas suposiciones:

* Solo se clasificarán dos variables por ahora
* `cov_est` será concat
* `norm_trace` es falso

Eliminaremos también ciertas validaciones.

In [ ]:
    def fit(self, X, y):
        """Estimate the CSP decomposition on epochs.
        Parameters
        ----------
        X : ndarray, shape (n_epochs, n_channels, n_times)
            The data on which to estimate the CSP.
        y : array, shape (n_epochs,)
            The class for each epoch.
        Returns
        -------
        self : instance of CSP
            Returns the modified instance.
        """
        n_channels = X.shape[1]

        self._classes = np.unique(y)
        n_classes = len(self._classes)

        covs = np.zeros((n_classes, n_channels, n_channels))
        # sample_weights = list() # multiclass
        
        for class_idx, this_class in enumerate(self._classes):
            
            if self.cov_est == "concat":  # concatenate epochs
                class_ = np.transpose(X[y == this_class], [1, 0, 2])
                class_ = class_.reshape(n_channels, -1)
                cov = _regularized_covariance(
                    class_, reg=self.reg, method_params=self.cov_method_params,
                    rank=self.rank)
                weight = sum(y == this_class)

            covs[class_idx] = cov
            # sample_weights.append(weight) # multiclass

        if n_classes == 2:
            eigen_values, eigen_vectors = linalg.eigh(covs[0], covs.sum(0))
            # sort eigenvectors
            ix = np.argsort(np.abs(eigen_values - 0.5))[::-1]

        # sort eigenvectors
        eigen_vectors = eigen_vectors[:, ix]

        self.filters_ = eigen_vectors.T
        self.patterns_ = linalg.pinv2(eigen_vectors)

        pick_filters = self.filters_[:self.n_components]
        X = np.asarray([np.dot(pick_filters, epoch) for epoch in X])

        # compute features (mean band power)
        X = (X ** 2).mean(axis=2)

        # To standardize features
        self.mean_ = X.mean(axis=0)
        self.std_ = X.std(axis=0)

        return self

## Objetivo

El objetivo principal es conseguir los patrones y los filtros a partir de $X$, $y$. O sea estas líneas de acá:

        self.filters_ = eigen_vectors.T
        self.patterns_ = linalg.pinv2(eigen_vectors)

Eso quiere decir que tenemos que hallar los autovectores de algo... que es la matriz de covarianza.

        cov = _regularized_covariance(
                    class_, reg=self.reg, method_params=self.cov_method_params,
                    rank=self.rank) 
                    
Y esta matriz la sacamos de... $X$, solo que hay que modificarlo un poquito. Pero en todo caso el pipeline simplificado es el siguiente:

1. Transformar $X$ a $X'$
2. Hallar la matriz de covarianza de $X'$
3. Hallar los autovectores a partir de $C_X$

$y$ es importante porque no vamos a hallar solo una matriz de covarianza, sino varias (el número de clases únicas en $y$). Esto último creo que solo es necesario para las clasificación multiclass.

### 1. Transformar $X$ a $X'$

Para entender esta parte del código es importante entender la representación de la data. Tres transformaciones principales sufren los datos desde que se obtienen hasta que son usados por el algoritmo CSP.

Primero, la data es guardada en una matriz canales $\times$ tiempo. En esta matriz están concatenadas todas las sesiones de todos los movimientos, pero nosotros hemos anotado los tiempos y clases de antemano. El siguiente sería un ejemplo de las ondas cerebrales concatenadas y con anotaciones:

In [11]:
# imagen del otro cuaderno

Para trabajar en general con esta información, primero se separa la información en epochs. Los epochs son ventanas de tiempo que también son matrices canales $\times$ tiempo. La diferencia es que tienen guardado el tiempo exacto en el cual se ejecutó el movimiento. Todos estos epochs se guardan en un gran arreglo, y ahí viene la primera gran transformación:

$$ channels \times samples \rightarrow epochs \times channels \times samples $$

Samples viene a ser lo mismo que el tiempo.

## Definiendo variables

$X$ e $y$ son los inputs. $X$ es la matriz de epochs, donde cada epoch es una matriz de channels $\times$ samples (epochs $\times$ channels $\times$ samples). $y$, por otro lado, es un arreglo con las clases de cada epoch.

Otros datos que se necesitan son:
* Número de canales (que se obtiene de $X$)
* Número de clases (que se obtiene de $y$)
* Una matriz de covarianza (que es classes $\times$ channels $\times$ channels y no sé por qué)
* Una lista con los pesos de las muestras o sample (supongo que cada epoch debe tener un número igual de samples para que funcione)

Otros datos que se necesitan son (v2):
* Número de canales (que se obtiene de $X$)
* Número de clases (que se obtiene de $y$)
* Una matriz de covarianza (que es classes $\times$ channels $\times$ channels)
* Una lista con los pesos de las muestras o sample (pesos por clase)

In [ ]:
self._classes = np.unique(y)

n_channels = X.shape[1]
n_classes = len(self._classes)

covs = np.zeros((n_classes, n_channels, n_channels))
sample_weights = list()

# Calculando la matriz de covarianza por clase

Supongo que la matriz de convarianza es de channels $\times$ channels, o 14 $\times$ 14.

In [ ]:
for class_idx, this_class in enumerate(self._classes):
    class_ = np.transpose(X[y == this_class], [1, 0, 2])
    class_ = class_.reshape(n_channels, -1)
    cov = _regularized_covariance(class_, reg=self.reg, method_params=self.cov_method_params, rank=self.rank)
    weight = sum(y == this_class)

    covs[class_idx] = cov

Vamos a detenernos en esta línea. Se seleccionan solo los epochs pertenecientes a esa clase con `X[y == this_class]`. Y luego se transpone de tal forma que, antes:

* Teníamos los epochs, y dentro de cada epoch teníamos todos los canales

Y ahora:

* Tenemos todos los canales, y dentro de cada uno los epochs

Si la finalidad es convertir una matriz de channels $\times$ samples a una de sources $\times$ samples, yo diría que es el camino correcto... creo.

In [ ]:
class_ = np.transpose(X[y == this_class], [1, 0, 2])

In [2]:
import numpy as np

# epochs × channels × samples
epochs = 5 # 5 epochs of 1 seg each
channels = 5 # Insight
samples = 10 # 10Hz
X = np.zeros((epochs, channels, samples))

for i in range(0, len(X)):
    X[i][0][0] = i + 1
    
X

array([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[2., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[3., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[4., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [3]:
y = np.random.randint(0, 2, size=epochs)
y

array([0, 1, 0, 1, 0])

In [7]:
weight = sum(y == 1)
weight

2

In [9]:
weight = sum(y == 0)
weight

3

In [10]:
y == 0

array([ True, False,  True, False,  True])

In [4]:
X[y == 1] # epochs x channels x samples

array([[[2., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[4., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])

In [5]:
class_ = np.transpose(X[y == 1], [1, 0, 2]) # channels x epochs x samples
class_

array([[[2., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [4., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])

Aquí estoy concatenando los epochs por channel

In [6]:
class_ = class_.reshape(channels, -1)
class_

array([[2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

Hasta aquí llegué, necesito revisar las cosas de algebra lineal

In [33]:
cov = _regularized_covariance(class_, reg=self.reg, method_params=self.cov_method_params, rank=self.rank)

NameError: name '_regularized_covariance' is not defined

In [ ]:
def _regularized_covariance(data, reg=None, method_params=None, info=None,
                            rank=None):
    """Compute a regularized covariance from data using sklearn.
    This is a convenience wrapper for mne.decoding functions, which
    adopted a slightly different covariance API.
    Returns
    -------
    cov : ndarray, shape (n_channels, n_channels)
        The covariance matrix.
    """
    if reg is None:
        reg = 'empirical'
    try:
        reg = float(reg)
    except ValueError:
        pass
    if isinstance(reg, float):
        if method_params is not None:
            raise ValueError('If reg is a float, method_params must be None '
                             '(got %s)' % (type(method_params),))
        method_params = dict(shrinkage=dict(
            shrinkage=reg, assume_centered=True, store_precision=False))
        reg = 'shrinkage'
    elif not isinstance(reg, str):
        raise ValueError('reg must be a float, str, or None, got %s (%s)'
                         % (reg, type(reg)))
    method, method_params = _check_method_params(
        reg, method_params, name='reg', allow_auto=False, rank=rank)
    # use mag instead of eeg here to avoid the cov EEG projection warning
    info = create_info(data.shape[-2], 1000., 'mag') if info is None else info
    picks_list = _picks_by_type(info)
    scalings = _handle_default('scalings_cov_rank', None)
    cov = _compute_covariance_auto(
        data.T, method=method, method_params=method_params,
        info=info, cv=None, n_jobs=1, stop_early=True,
        picks_list=picks_list, scalings=scalings,
        rank=rank)[reg]['data']
    return cov